# PhraseListTagger
## Example 1

In [1]:
from estnltk import Text
text = Text('Patsient lasi jalga, sest vasaku jala suure varba pika \
            painutajalihase kõõluse rebend ajas tal kopsu üle maksa.')
text.analyse('morphology')

Text(text="Patsient lasi jalga, sest vasaku jala suure varba pika             painutajalihase kõõluse rebend ajas tal kopsu üle maksa.")

In [2]:
from estnltk.finite_grammar import PhraseListTagger

phrase_list = [
               ('jalg'),
               ('vasak', 'jalg',),
               ('parem', 'jalg',),
               ('kops',),
               ('vasak', 'kops'),
               ('parem', 'kops'),
               ('kõõlus',),
               ('lihas',),
               ('maks',),
               ('neer',),
               ('parem', 'neer'),
               ('vasak', 'neer'),
               ('varvas',),
               ('suur', 'varvas'),
              ]

def decorator(text, span, phrase):
    return {'match': phrase, 'dfsd':None}

def consistency_checker(text, span, phrase):
    return True

tagger = PhraseListTagger(layer_name='body_parts',
                          input_layer='morph_analysis',
                          input_attribute='lemma',
                          phrase_list=phrase_list,
                          attributes=('match',),
                          decorator=decorator,
                          consistency_checker=consistency_checker,
                          conflict_resolving_strategy= 'MAX')
tagger

In [3]:
tagger.tag(text)

Text(text="Patsient lasi jalga, sest vasaku jala suure varba pika             painutajalihase kõõluse rebend ajas tal kopsu üle maksa.")

In [4]:
text['body_parts']

Note that 'maksa' is not tagged. This is because Vabamorf thinks its lemma is 'maksma'.

## Example 2: postprocessing
Suppose that we want to postprocess the layer before adding it to the text object. Here we add an attribute 'latin_term' to the layer.

In [5]:
text = Text('Vasaku jala suure varba pika painutajalihase kõõluse rebend ajas patsiendi kopsu üle maksa.')
text.analyse('morphology')
latin_dict = {('suur', 'varvas'):'hallux', ('kõõlus',):'tendo', ('kops',):'pulmo'}

layer = tagger.tag(text, return_layer=True)
layer.attributes = ('match','latin_term')
for s in layer:
    s.latin_term = latin_dict[s.match]
text['body_parts'] = layer
text['body_parts']

## Example 3

In [6]:
text = Text('Naeris naeris.')
text.analyse('morphology')

Text(text="Naeris naeris.")

Create one more layer of morphological analysis.

In [7]:
from estnltk.taggers import VabamorfTagger
tagger = VabamorfTagger(layer_name='morph_analysis_no_disambiguate',
                        disambiguate=False)
tagger.tag(text)

Text(text="Naeris naeris.")

In [8]:
from estnltk.resolve_layer_dag import make_resolver

def decorator(text, span, phrase):
    return {'match': phrase}

tagger = PhraseListTagger('test',
                          'morph_analysis_no_disambiguate',
                          'lemma',
                          [
                            ('naerma', 'naerma'),
                            ('naerma', 'naeris'),
                            ('naeris', 'naerma'),
                            ('naeris', 'naeris'),
                          ],
                          attributes=('match',),
                          decorator=decorator,
                          conflict_resolving_strategy='ALL')
tagger.tag(text)
text['test']

# Millised peaksid olema decorator'i, consistency_checker'i argumendid?

Näiteks olgu meil mitmene morph_analysis kiht ja matchime lemmasid ('naeris', 'naerma'). Kui anname ette mitmese spanni (seitsmene spanlist + neljane spanlist), siis ei ole üheselt selge milliseid lemmasid õnnestus matchida. Kui anname ette konkreetsed spanid, mis matchisid, näiteks kuuenda ja esimese, siis pole kindel, et parem poleks olnud ette anda hoopis seitsmendat ja esimest.

In [9]:
text.morph_analysis_no_disambiguate

SL[SL[Span(Naeris, {'ending': 's', 'partofspeech': 'H', 'form': 'sg in', 'root_tokens': ('Naer',), 'clitic': '', 'lemma': 'Naer', 'root': 'Naer'}),
Span(Naeris, {'ending': 's', 'partofspeech': 'H', 'form': 'sg in', 'root_tokens': ('Naeri',), 'clitic': '', 'lemma': 'Naeri', 'root': 'Naeri'}),
Span(Naeris, {'ending': '0', 'partofspeech': 'H', 'form': 'sg n', 'root_tokens': ('Naeris',), 'clitic': '', 'lemma': 'Naeris', 'root': 'Naeris'}),
Span(Naeris, {'ending': 's', 'partofspeech': 'H', 'form': 'sg in', 'root_tokens': ('Naeris',), 'clitic': '', 'lemma': 'Naeris', 'root': 'Naeris'}),
Span(Naeris, {'ending': 'is', 'partofspeech': 'V', 'form': 's', 'root_tokens': ('naer',), 'clitic': '', 'lemma': 'naerma', 'root': 'naer'}),
Span(Naeris, {'ending': '0', 'partofspeech': 'S', 'form': 'sg n', 'root_tokens': ('naeris',), 'clitic': '', 'lemma': 'naeris', 'root': 'naeris'}),
Span(Naeris, {'ending': 's', 'partofspeech': 'S', 'form': 'sg in', 'root_tokens': ('naeris',), 'clitic': '', 'lemma': 'naeris', 'root': 'naeris'})],
SL[Span(naeris, {'ending': 'is', 'partofspeech': 'V', 'form': 's', 'root_tokens': ('naer',), 'clitic': '', 'lemma': 'naerma', 'root': 'naer'}),
Span(naeris, {'ending': '0', 'partofspeech': 'S', 'form': 'sg n', 'root_tokens': ('naeris',), 'clitic': '', 'lemma': 'naeris', 'root': 'naeris'}),
Span(naeris, {'ending': 's', 'partofspeech': 'S', 'form': 'sg in', 'root_tokens': ('naeris',), 'clitic': '', 'lemma': 'naeris', 'root': 'naeris'})],
SL[Span(., {'ending': '', 'partofspeech': 'Z', 'form': '', 'root_tokens': ('.',), 'clitic': '', 'lemma': '.', 'root': '.'})]]